In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler


df = pd.read_excel("ML470_S9_KCHouse_Data_Practice.xlsx")


